In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 3), abs(state[i*4+1] + state[i*4+3] * 3), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(3)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.9
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action)
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0

print("Training finished.\n")

Episode: 100
average reward: 187.07
Episode: 200
average reward: 294.95
Episode: 300
average reward: 334.32
Episode: 400
average reward: 388.02
Episode: 500
average reward: 395.59
Episode: 600
average reward: 403.76


In [6]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [7]:
import time
total_epochs, total_penalties = 0, 0
episodes = 10
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 10 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 108.6
Average reward per episode: 28.0
